In [1]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import pandas as pd 
import inspect
import sys, logging

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)




In [2]:
def eval_scm(scm, nsamples, **kwargs):
    values = dict(N = nsamples)
    values.update(kwargs)
    
    for varname, func in scm.items():
        argnames = list(inspect.signature(func).parameters.keys())
        logging.info(f"{varname} : f({argnames})")
        args = { name : values[name] for name in argnames }
        values[varname] =func(**args)
    return values

In [7]:
class CondMeanDiff:

    def __init__(self, treatment, response):
        self.treatment_name = treatment
        self.response_name = response

    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        return response[treatment >= 0.5].mean() - response[treatment < 0.5].mean()    


class ATEStratified:
    def __init__(self, treatment, response, strat_func):
        self.treatment_name = treatment
        self.response_name = response
        self.strat_func = strat_func        

    def __call__(self, values):        
        response = values[self.response_name]
        treatment = values[self.treatment_name]

        keys = self.strat_func(values)
        unique_keys =set(keys.tolist())
        ate = 0.0
        total_weight = 0
        for k in unique_keys:        
            is_treatment = (treatment > 0.5)
            treatment_mask = (keys == k) & is_treatment
            control_mask = (keys == k) & (~is_treatment)
            if (treatment_mask.sum() == 0 or control_mask.sum() == 0):
                logging.info(f"skipping group {k}")
                continue
            group_ate = (response[treatment_mask].mean() - response[control_mask].mean())
            group_weight = ((keys == k)*1).sum()
            total_weight += group_weight
            ate = ate + group_ate*group_weight
        ate = ate / total_weight
        return ate


class SplitStratify:
    def __init__(self, varname, splits):
        self.varname = varname
        self.splits = torch.Tensor(splits)

    def __call__(self, values):
        vals = values[self.varname]
        keys = torch.zeros(vals.shape)
        for bar in self.splits:
            keys += (vals < bar )
        return keys

class QuantStratify:
    def __init__(self, varname, ngroups):
        self.varname = varname
        self.ngroups = ngroups

    def __call__(self, values):
        vals = values[self.varname]
        keys = torch.zeros(vals.shape)
        quantiles = torch.quantile(vals, torch.arange(self.ngroups)[1:]/float(self.ngroups))
        for q in quantiles:
            keys += (vals <  q)
        return keys

def enrich_propensity(values, target_name, covariate_names, propensity_name="propensity", model=None):
    # Convert target to numpy
    y = values[target_name].cpu().numpy()
    if y.ndim > 1 and y.shape[1] == 1:
        y = y.flatten()

    # Concatenate covariates (converted to numpy)
    X = torch.stack([values[name] for name in covariate_names]).numpy().transpose()    
    # Create and fit sklearn logistic regression
    if model is None:
        model = LogisticRegression(max_iter=1000)
    logging.info(f"fitting model for propensity: '{propensity_name}'")
    model.fit(X, y)

    # Predict probabilities (for positive class)
    probs = model.predict_proba(X)[:, 1]

    # Convert back to torch tensor
    probs_tensor = torch.from_numpy(probs).float()
    values[propensity_name] = probs_tensor

class ATEPropensity:

    def __init__(self, treatment, response, propensity):
        self.treatment_name = treatment
        self.response_name = response
        self.propensity_name = propensity

    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        prop = values[self.propensity_name]
        
        treatment_mask = (treatment >= 0.5)
        
        return (
            (response[treatment_mask]/prop[treatment_mask]).sum() / (1.0/prop[treatment_mask]).sum() -
            (response[~treatment_mask]/(1-prop[~treatment_mask])).sum() / (1.0/(1-prop[~treatment_mask])).sum()
        )

class ATEImpute:

    def __init__(self, treatment, response, conditions, mgen=None):
        self.treatment_name = treatment
        self.response_name = response
        self.condition_names = conditions
        self.mgen = mgen or (lambda: GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3))
        
    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        
        y = response.numpy()

        # Concatenate covariates (converted to numpy)
        X = torch.stack([values[name] for name in [self.treatment_name] + self.condition_names]).numpy().transpose()    
        model = self.mgen()
        logging.info(f"fitting model for imputation")
        model.fit(X, y)

        treatment_mask = (treatment > 0.5)
        treatment_counterfactual_covs = X[(treatment_mask.numpy()), :]        
        treatment_counterfactual_covs[:,0] = 0
        
        treatment_counterfactual_resp = torch.from_numpy(model.predict(treatment_counterfactual_covs)).float()        

        control_mask = ~treatment_mask
        control_counterfactual_covs = X[(control_mask.numpy()), :]
        control_counterfactual_covs[:,0] = 1
        control_counterfactual_resp = torch.from_numpy(model.predict(control_counterfactual_covs)).float()        

        ate = (
            (response[treatment_mask] -  treatment_counterfactual_resp).sum() + 
            (control_counterfactual_resp - response[control_mask]).sum()) / response.numel()
        return ate
        


In [8]:
def show(**kwargs):
    return pd.DataFrame({k:v.unsqueeze(0) for k,v in kwargs.items()})

## Sanity - No causality
(Two items of the same gerne)

In [9]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.2 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.2 + 0.5*U))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))



2025-07-18 15:18:40,133 - INFO - U : f(['N'])
2025-07-18 15:18:40,136 - INFO - T : f(['N', 'U'])
2025-07-18 15:18:40,138 - INFO - Y : f(['N', 'U', 'T'])
2025-07-18 15:18:40,140 - INFO - fitting model for propensity: 'propensity'
2025-07-18 15:18:40,206 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.262071,0.012124,0.012185,0.012127


## Causality with Confounder

In [10]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.5*U + 0.2*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-18 15:18:44,576 - INFO - U : f(['N'])
2025-07-18 15:18:44,578 - INFO - T : f(['N', 'U'])
2025-07-18 15:18:44,580 - INFO - Y : f(['N', 'U', 'T'])
2025-07-18 15:18:44,582 - INFO - fitting model for propensity: 'propensity'
2025-07-18 15:18:44,661 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.47532,0.200034,0.200134,0.200035


## Causality with a weak Confounder
(For example Godfather-I, Godfather II)

In [11]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.05 + 0.05*U  + 0.5*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))


show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-18 15:18:46,586 - INFO - U : f(['N'])
2025-07-18 15:18:46,588 - INFO - T : f(['N', 'U'])
2025-07-18 15:18:46,590 - INFO - Y : f(['N', 'U', 'T'])
2025-07-18 15:18:46,593 - INFO - fitting model for propensity: 'propensity'
2025-07-18 15:18:46,670 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.526907,0.500818,0.500827,0.500811


## Information Leakage

In [12]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.4*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.4*U  + 0.5*T))*1.0,
    "Leakage" : lambda N, T, Y: (T+Y) * (torch.rand(N) < 0.3) 
}

values = eval_scm(SCM, 10000)
enrich_propensity(values, "T", ["U"], "propensity", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))
enrich_propensity(values, "T", ["U","Leakage"], "propensity_leakage", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

mgen = lambda: GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"], mgen=mgen)(values),
     ATE_Propensity_Leakage=ATEPropensity("T","Y", "propensity_leakage")(values),
     ATE_Impute_Leakage=ATEImpute("T","Y",["U","Leakage"], mgen=mgen)(values))
     


2025-07-18 15:18:47,500 - INFO - U : f(['N'])
2025-07-18 15:18:47,503 - INFO - T : f(['N', 'U'])
2025-07-18 15:18:47,506 - INFO - Y : f(['N', 'U', 'T'])
2025-07-18 15:18:47,509 - INFO - Leakage : f(['N', 'T', 'Y'])
2025-07-18 15:18:47,509 - INFO - fitting model for propensity: 'propensity'
2025-07-18 15:18:47,715 - INFO - fitting model for propensity: 'propensity_leakage'
2025-07-18 15:18:47,987 - INFO - fitting model for imputation
2025-07-18 15:18:48,106 - INFO - fitting model for imputation


,Naive,ATE_Propensity,ATE_Impute,ATE_Propensity_Leakage,ATE_Impute_Leakage
0,0.68469,0.494835,0.494822,0.496154,0.42762


## Simple Symetric BiDirectional Causality

We generate the variables for FirstMovie and SecondMovie (causality is always from First to Second),
Then decide if MovieA is first, or whether there was a swap

In [13]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "FirstMovie" : lambda N, U: (torch.rand(N) < (0.1 + 0.4*U))*1.0,
    "SecondMovie" : lambda N, U, FirstMovie: (torch.rand(N) < (0.1 + 0.3*U + 0.4*FirstMovie))*1.0,
    "Swap" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "MovieA": lambda Swap, FirstMovie, SecondMovie: torch.where(Swap < 0.5, FirstMovie, SecondMovie),
    "MovieB": lambda Swap, FirstMovie, SecondMovie: torch.where(Swap < 0.5, SecondMovie, FirstMovie),
    "Y" : lambda MovieB: MovieB,
    "T" : lambda Swap, MovieA:  ((Swap < 0.5) & (MovieA > 0.5)) * 1.0
}

values = eval_scm(SCM, 100000)

enrich_propensity(values, "T", ["U"])

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))



2025-07-18 15:18:48,895 - INFO - U : f(['N'])
2025-07-18 15:18:48,899 - INFO - FirstMovie : f(['N', 'U'])
2025-07-18 15:18:48,901 - INFO - SecondMovie : f(['N', 'U', 'FirstMovie'])
2025-07-18 15:18:48,903 - INFO - Swap : f(['N'])
2025-07-18 15:18:48,904 - INFO - MovieA : f(['Swap', 'FirstMovie', 'SecondMovie'])
2025-07-18 15:18:48,906 - INFO - MovieB : f(['Swap', 'FirstMovie', 'SecondMovie'])
2025-07-18 15:18:48,908 - INFO - Y : f(['MovieB'])
2025-07-18 15:18:48,908 - INFO - T : f(['Swap', 'MovieA'])
2025-07-18 15:18:48,908 - INFO - fitting model for propensity: 'propensity'
2025-07-18 15:18:48,982 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.487463,0.362627,0.362703,0.362621
